In [1]:
#
# IE 678 Deep Learning, University of Mannheim
# Author: Rainer Gemulla

In [2]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from IPython import get_ipython
from util import nextplot

%matplotlib notebook
get_ipython().magic('run -i "a01-fnn-helper.py"')

# 1 Perceptrons

In [ ]:
# plot X1 (separable)
nextplot()
plot2(X1, y1)

In [ ]:
# plot X2 (not separable)
nextplot()
plot2(X2, y2)

In [ ]:
def pt_classify(X, w):
    """Classify using a perceptron.

    Parameters
    ----------
    X : torch array of shape (N,D) or shape (D,)
        Design matrix of test examples
    w : torch array of shape (D,)
        Weight vector

    Returns
    -------
    torch array of shape (N,)
        Predicted binary labels (either 0 or 1)"""
    if X.dim() == 1:
        X = X.view(1, -1)
    return (X @ w >= 0).int()

## 1a+c Learning

In [ ]:
def pt_train(X, y, maxepochs=100, pocket=False, w0=None):
    """Train a perceptron.

    Parameters
    ----------
    X : torch array of shape (N,D)
        Design matrix
    y : torch array of shape (N,)
        Binary labels (either 0 or 1)
    maxepochs : int
        Maximum number of passes through the training set before the algorithm
        returns
    pocket : bool
       Whether to use the pocket algorithm (True) or the perceptron learning algorithm
       (False)
    w0 : torch array of shape (D,)
        Initial weight vector

    Returns
    -------
    torch array of shape (D,)
        Fitted weight vector"""

    N, D = X.shape
    if w0 is None:  # initial weight vector
        w0 = torch.zeros(D)
    w = w0  # current weight vector

    ## YOUR CODE HERE

## 1b+d Experimentation

In [ ]:
# Train a perceptron using the perceptron learning algorithm and plot decision
# boundary. You should get a perfect classification here. The decision boundary
# should not change if you run this multiple times.
w = pt_train(X1, y1)
nextplot()
plot2(X1, y1)
plot2db(w, label="perceptron")

In [ ]:
# Train a perceptron using the pocket algorithm and plot decision boundary. You
# should get a perfect classification here (with high probability). The decision
# boundary should change if you run this multiple times.
w = pt_train(X1, y1, pocket=True)
nextplot()
plot2(X1, y1)
plot2db(w, label="pocket")

In [ ]:
# Train 10 perceptrons starting with random weights. Also train logistic
# regression and SVM. Plot all decision boundaries, and print the
# misclassification rates (on training data). Try this with and without the
# pocket algorithm.
nextplot()
plot2dbs(X2, y2, n=10, maxepochs=1000, pocket=True)

# 2 Multi-Layer Feed-Forward Neural Networks

## 2a Conjecture how an FNN fit will look like

In [ ]:
# here is the one-dimensional dataset that we will use
nextplot()
plot1(X3, y3, label="train")
plot1(X3test, y3test, label="test")
plt.legend()

## 2b Train with 2 hidden units

In [ ]:
# Training code. You do not need to modify this code.
train_bfgs = lambda model, **kwargs: train_scipy(X3, y3, model, **kwargs)


def train3(
    hidden_sizes, nreps=10, transfer=lambda: nn.Sigmoid(), train=train_bfgs, **kwargs
):
    """Train an FNN.

    hidden_sizes is a (possibly empty) list containing the sizes of the hidden layer(s).
    nreps refers to the number of repetitions.

    """
    best_model = None
    best_cost = math.inf
    for rep in range(nreps):
        model = fnn_model([1] + hidden_sizes + [1], transfer)
        print(f"Repetition {rep: 2d}: ", end="")
        model = train(model, **kwargs)
        mse = F.mse_loss(y3, model(X3)).item()
        if mse < best_cost:
            best_model = model
            best_cost = mse
        print(f"best_cost={best_cost:.3f}")

    return best_model

In [ ]:
# Let's fit the model with one hidden layer consisting of 2 units.
model = train3([2], nreps=1)
print("Training error:", F.mse_loss(y3, model(X3)).item())
print("Test error    :", F.mse_loss(y3test, model(X3test)).item())

In [ ]:
# plot the data and the fit
nextplot()
plot1(X3, y3, label="train")
plot1(X3test, y3test, label="test")
plot1fit(torch.linspace(0, 13, 500).unsqueeze(1), model)

In [ ]:
# The weight matrices and bias vectors can be read out as follows. If you want,
# use these parameters to compute the output of the network (on X3) directly and
# compare to model(X3).
for par, value in model.state_dict().items():
    print(f"{par:<15}= {value}")

In [ ]:
# now repeat this multiple times
# YOUR CODE HERE

In [ ]:
# From now on, always train multiple times (nreps=10 by default) and
# report best model.
model = train3([2], nreps=10)
print("Training error:", F.mse_loss(y3, model(X3)).item())
print("Test error    :", F.mse_loss(y3test, model(X3test)).item())

## 2c Width

In [ ]:
# Experiment with different hidden layer sizes. To avoid recomputing
# models, you may want to save your models using torch.save(model, filename) and
# load them again using torch.load(filename).
# YOUR CODE HERE

## 2d Distributed representations

In [ ]:
# train a model to analyze
model = train3([2])

In [ ]:
# plot the fit as well as the outputs of each neuron in the hidden
# layer (scale for the latter is shown on right y-axis)
nextplot()
plot1(X3, y3, label="train")
plot1(X3test, y3test, label="test")
plot1fit(torch.linspace(0, 13, 500).unsqueeze(1), model, hidden=True, scale=False)

In [ ]:
# plot the fit as well as the outputs of each neuron in the hidden layer, scaled
# by its weight for the output neuron (scale for the latter is shown on right
# y-axis)
nextplot()
plot1(X3, y3, label="train")
plot1(X3test, y3test, label="test")
plot1fit(torch.linspace(0, 13, 500).unsqueeze(1), model, hidden=True, scale=True)

## 2e Experiment with different optimizers (optional)

In [ ]:
# PyTorch provides many gradient-based optimizers; see
# https://pytorch.org/docs/stable/optim.html. You can use a PyTorch optimizer
# as follows.
train_adam = lambda model, **kwargs: fnn_train(
    X3, y3, model, optimizer=torch.optim.Adam(model.parameters(), lr=0.01), **kwargs
)
model = train3([50], nreps=1, train=train_adam, max_epochs=5000, tol=1e-8, verbose=True)

In [ ]:
# Experiment with different number of layers and activation functions. Here is
# an example with three hidden layers (of sizes 4, 5, and 6) and ReLU activations.
#
# You can also plot the outputs of the hidden neurons in the first layer (using
# the same code above).
model = train3([4, 5, 6], nreps=50, transfer=lambda: nn.ReLU())
nextplot()
plot1(X3, y3, label="train")
plot1(X3test, y3test, label="test")
plot1fit(torch.linspace(0, 13, 500).unsqueeze(1), model)
print("Training error:", F.mse_loss(y3, model(X3)).item())
print("Test error    :", F.mse_loss(y3test, model(X3test)).item())